In [1]:
import scraper as s
import pandas as pd
import numpy as np
import datetime

In [2]:
m = s.stockMongo()
apple = m.get_options("AAPL")

In [3]:
apple

,ask,bid,contract,in-money,iv,last,last_trade,strike,strike-date,type,volume
date,,,,,,,,,,,
2020-08-02 23:39:23.470,247.10,242.90,AAPL200807C00180000,in-the-money,379.98%,243.38,2020-07-31,180.0,2020-08-07,Call,3.0
2020-08-02 23:39:23.470,241.45,238.30,AAPL200807C00185000,in-the-money,344.78%,239.60,2020-07-31,185.0,2020-08-07,Call,15.0
2020-08-02 23:39:23.470,236.60,233.30,AAPL200807C00190000,in-the-money,340.72%,233.35,2020-07-31,190.0,2020-08-07,Call,13.0
2020-08-02 23:39:23.470,222.50,218.00,AAPL200807C00205000,in-the-money,233.01%,171.98,2020-07-30,205.0,2020-08-07,Call,NaN
2020-08-02 23:39:23.470,217.50,213.00,AAPL200807C00210000,in-the-money,225.78%,166.97,2020-07-30,210.0,2020-08-07,Call,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2020-08-14 20:02:22.116,133.50,130.45,AAPL220916P00520000,in-the-money,36.00%,132.14,2020-08-14,520.0,2022-09-16,Put,20.0
2020-08-14 20:02:22.116,147.05,143.80,AAPL220916P00540000,in-the-money,35.81%,142.00,2020-08-13,540.0,2022-09-16,Put,2.0
2020-08-14 20:02:22.116,161.60,158.10,AAPL220916P00560000,in-the-money,35.81%,161.93,2020-08-14,560.0,2022-09-16,Put,21.0


In [4]:
#apple = apple.where(apple['last_trade']==apple.index.date).dropna()
apple = apple.drop(['volume', 'iv', 'in-money', 'bid', 'ask','last_trade', 'contract'], axis=1)

In [5]:
apple = apple.reset_index()
apple = apple.set_index(['strike-date', 'type'])
apple['date'] = pd.to_datetime(apple['date']).dt.strftime('%m-%d-%Y')
apple['date'] = pd.to_datetime(apple['date'])
apple.head()


date    last  strike
strike-date type                           
2020-08-07  Call 2020-08-02  243.38   180.0
            Call 2020-08-02  239.60   185.0
            Call 2020-08-02  233.35   190.0
            Call 2020-08-02  171.98   205.0
            Call 2020-08-02  166.97   210.0

In [7]:
strike_pivot = apple.pivot_table(columns="strike-date", values="date", aggfunc=np.count_nonzero)
strike_cols = strike_pivot.columns
strike_cols



DatetimeIndex(['2020-08-07', '2020-08-14', '2020-08-21', '2020-08-28',
               '2020-09-04', '2020-09-18', '2020-10-16', '2020-11-20',
               '2020-12-18', '2021-01-15', '2021-06-18', '2021-09-17',
               '2022-01-21', '2022-06-17', '2022-09-16'],
              dtype='datetime64[ns]', name='strike-date', freq=None)

In [8]:
#aCLo = apple.loc[('2020-09-04')]
apl = m.get_finnhub_prices('AAPL', min(apple['date']), max(apple['date']))
apl['date'] = apl.index
apl = apl.reset_index()
apl = apl.drop(['h', 'l', 'o', 'v','t'], axis=1)

In [9]:
new_apple = []
apple2 = apple
#for cols in strike_cols:
cols = strike_cols[0]
apple2 = apple2.reset_index()
apple2 = apple2.set_index(['type','strike-date'])
apple_pivot_call = apple2.loc[('Call', cols)]
strike_pivot = apple_pivot_call.pivot_table(columns="strike", values="date", aggfunc=np.count_nonzero)
strike_value_cols = strike_pivot.columns
apple_pivot_put = apple2.loc[('Put', cols)]
strike_pivot = apple_pivot_put.pivot_table(columns="strike", values="date", aggfunc=np.count_nonzero)
strike_value_puts = strike_pivot.columns
    
apple2 = apple2.reset_index()
apple2 = apple2.set_index(['type', 'strike','strike-date'])

for i in range(len(strike_value_cols)-1):
    for j in range(len(strike_value_puts)-1):
        aCLo = apple2.loc[('Call', strike_value_cols[i], cols)]
        aCLo = aCLo.reset_index()
        #aCLo = aCLo.drop(['type', 'strike-date'], axis=1)
        aCLo.rename(columns = {'last':'priceCLo', 'strike': 'strikeCLo'}, inplace = True)
        aCHi = apple2.loc[('Call', strike_value_cols[i+1], cols)]
        aCHi = aCHi.reset_index()
        #aCHi = aCHi.drop(['type', 'strike-date'], axis=1)
        aCHi.rename(columns = {'last':'priceCHi', 'strike': 'strikeCHi'}, inplace = True)
        aPLo = apple2.loc[('Put', strike_value_puts[j], cols)]
        aPLo = aPLo.reset_index()
        #aPLo = aPLo.drop(['type', 'strike-date'], axis=1)
        aPLo.rename(columns = {'last':'pricePLo', 'strike': 'strikePLo'}, inplace = True)
        aPHi = apple2.loc[('Put', strike_value_puts[j+1], cols)]
        aPHi = aPHi.reset_index()
        #aPHi = aPHi.drop(['type', 'strike-date'], axis=1)
        aPHi.rename(columns = {'last':'pricePHi', 'strike': 'strikePHi'}, inplace = True)

        a = pd.merge(aCLo, apl, on='date')
        a = pd.merge(a, aCHi, on='date')
        a = pd.merge(a, aPLo, on='date')
        a = pd.merge(a, aPHi, on='date')

        new_apple.append(a)


c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:7: PerformanceWarning: indexing past lexsort depth may impact performance.
  import sys
c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:10: PerformanceWarning: indexing past lexsort depth may impact performance.
  # Remove the CWD from sys.path while we load stuff.
c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:19: PerformanceWarning: indexing past lexsort depth may impact performance.
c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:23: PerformanceWarning: indexing past lexsort depth may impact performance.
c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:27: PerformanceWarning: indexing past lexsort depth may impact performance.
c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:31: PerformanceWarning: indexing past 

In [10]:
def iron_condor_price(x):
    s = -x['priceCHi']+x['priceCLo']+x['pricePHi']-x['pricePLo']
    
    return s/(x['strikePHi']-x['strikePLo'])

def iron_condor_value(x):
    s = 0
    if x['c'] >= x['strikeCLo']:
        s = s - x['c'] + x['strikeCLo']
    if x['c'] >= x['strikeCHi']:
        s = s + x['c'] - x['strikeCHi']
    if x['c'] <= x['strikePHi']:
        s = s - x['c'] + x['strikePHi']
    if x['c'] <= x['strikePLo']:
        s = s + x['c'] - x['strikePLo']
    #/(x['strikeP360']-x['strikeP350'])
    return s

In [11]:
a['daily_return'] = new_apple[0].apply(iron_condor_price, axis=1).pct_change(1)
a['value'] = new_apple[0].apply(iron_condor_value, axis=1)

In [36]:
a.loc[:,['strikeCLo', 'strike-date_x', 'date', 'c', 'strikeCHi', 'priceCHi', 'strikePLo', 'pricePLo', 'strikePHi', 'pricePHi']]
#a.head()

KeyError: 'Passing list-likes to .loc or [] with any missing labels is no longer supported, see https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike'

In [88]:
apple['time-left'] = apple['strike-date']-apple.index
apple[apple['time-left'].between(pd.Timedelta(1,'D'), pd.Timedelta(7,'D'))]

,last,strike,strike-date,type,time-left
date,,,,,
2020-06-12 18:01:24.609,4.74,327.5,2020-06-19,Put,6 days 05:58:35.391000
2020-06-12 18:01:24.609,5.60,330.0,2020-06-19,Put,6 days 05:58:35.391000
2020-06-12 18:01:24.609,6.55,332.5,2020-06-19,Put,6 days 05:58:35.391000
2020-06-12 18:01:24.609,7.50,335.0,2020-06-19,Put,6 days 05:58:35.391000
2020-06-12 18:01:24.609,8.73,337.5,2020-06-19,Put,6 days 05:58:35.391000
...,...,...,...,...,...
2020-07-15 20:01:35.342,139.25,515.0,2020-07-17,Put,1 days 03:58:24.658000
2020-07-15 20:01:35.342,120.60,520.0,2020-07-17,Put,1 days 03:58:24.658000
2020-07-15 20:01:35.342,131.70,530.0,2020-07-17,Put,1 days 03:58:24.658000


In [135]:
#apple = apple.reset_index()
#apple
b = apple.pivot_table(columns="strike-date", values="date", aggfunc=np.count_nonzero)
len(b.columns)


10

In [84]:
apple2 = apple2.reset_index()
apple2 = apple2.set_index(['type','strike-date'])
apple_pivot = apple2.loc[('Call', strike_cols[6])]

c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:3: PerformanceWarning: indexing past lexsort depth may impact performance.
  This is separate from the ipykernel package so we can avoid doing imports until


In [91]:
apple2

strike       date    last
type strike-date                           
Call 2021-09-17    110.0 2020-06-09  197.67
     2021-09-17    120.0 2020-06-09  122.30
     2021-09-17    125.0 2020-06-09  179.31
     2021-09-17    130.0 2020-06-09  201.75
     2021-09-17    135.0 2020-06-09  136.69
...                  ...        ...     ...
Put  2020-09-18    490.0 2020-08-01   98.05
     2020-09-18    495.0 2020-08-01  132.20
     2020-09-18    500.0 2020-08-01  177.56
     2020-09-18    545.0 2020-08-01  182.00
     2020-09-18    565.0 2020-08-01  154.36

[7231 rows x 3 columns]

In [106]:
apple2 = apple
apple2 = apple2.reset_index()
apple2 = apple2.set_index(['strike-date'])
strike_pivot = apple2.loc['Put'].pivot_table(columns="strike-date", values="date", aggfunc=np.count_nonzero)
strike_cols = strike_pivot.columns
strike_cols

DatetimeIndex(['2020-06-19', '2020-07-17', '2020-07-24', '2020-09-18',
               '2021-01-15', '2021-06-18', '2021-09-17', '2022-01-21',
               '2022-06-17', '2022-09-16'],
              dtype='datetime64[ns]', name='strike-date', freq=None)

In [4]:
apple2 = apple
apple2 = apple2.reset_index()
apple2 = apple2.set_index(['strike-date'])
apple2.loc['2020-08-07']

,date,ask,bid,contract,in-money,iv,last,last_trade,strike,type,volume
strike-date,,,,,,,,,,,
2020-08-07,2020-08-02 23:39:23.470,247.10,242.90,AAPL200807C00180000,in-the-money,379.98%,243.38,2020-07-31,180.0,Call,3.0
2020-08-07,2020-08-02 23:39:23.470,241.45,238.30,AAPL200807C00185000,in-the-money,344.78%,239.60,2020-07-31,185.0,Call,15.0
2020-08-07,2020-08-02 23:39:23.470,236.60,233.30,AAPL200807C00190000,in-the-money,340.72%,233.35,2020-07-31,190.0,Call,13.0
2020-08-07,2020-08-02 23:39:23.470,222.50,218.00,AAPL200807C00205000,in-the-money,233.01%,171.98,2020-07-30,205.0,Call,NaN
2020-08-07,2020-08-02 23:39:23.470,217.50,213.00,AAPL200807C00210000,in-the-money,225.78%,166.97,2020-07-30,210.0,Call,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2020-08-07,2020-08-02 23:39:23.470,98.00,93.90,AAPL200807P00520000,in-the-money,92.02%,146.55,2020-07-28,520.0,Put,NaN
2020-08-07,2020-08-02 23:39:23.470,107.95,103.40,AAPL200807P00530000,in-the-money,93.16%,154.75,2020-07-27,530.0,Put,NaN
2020-08-07,2020-08-02 23:39:23.470,118.00,113.40,AAPL200807P00540000,in-the-money,100.05%,134.65,2020-07-31,540.0,Put,6.0


In [16]:
a

,type_x,strikeCLo,strike-date_x,date,priceCLo,c,type_y,strikeCHi,strike-date_y,priceCHi,type_x,strikePLo,strike-date_x,pricePLo,type_y,strikePHi,strike-date_y,pricePHi
0,Call,530.0,2020-08-14,2020-08-04,0.32,438.66,Call,540.0,2020-08-14,0.25,Put,530.0,2020-08-14,92.60,Put,540.0,2020-08-14,103.80
1,Call,530.0,2020-08-14,2020-08-05,0.23,440.25,Call,540.0,2020-08-14,0.16,Put,530.0,2020-08-14,92.60,Put,540.0,2020-08-14,100.85
2,Call,530.0,2020-08-14,2020-08-06,0.42,455.61,Call,540.0,2020-08-14,0.29,Put,530.0,2020-08-14,78.53,Put,540.0,2020-08-14,83.95


In [22]:
new_apple[0]['']

,type_x,strikeCLo,strike-date_x,date,priceCLo,c,type_y,strikeCHi,strike-date_y,priceCHi,type_x,strikePLo,strike-date_x,pricePLo,type_y,strikePHi,strike-date_y,pricePHi
0,Call,180.0,2020-08-07,2020-08-04,259.10,438.66,Call,185.0,2020-08-07,239.6,Put,180.0,2020-08-07,0.01,Put,185.0,2020-08-07,0.45
1,Call,180.0,2020-08-07,2020-08-05,259.10,440.25,Call,185.0,2020-08-07,239.6,Put,180.0,2020-08-07,0.01,Put,185.0,2020-08-07,0.45
2,Call,180.0,2020-08-07,2020-08-06,275.92,455.61,Call,185.0,2020-08-07,272.7,Put,180.0,2020-08-07,0.01,Put,185.0,2020-08-07,0.45
